## Pairwise comparison

In [1]:
# this code may work only in command line
#install.packages(c("ggpubr","ggplot2","stringr"))

also installing the dependencies ‘Rcpp’, ‘matrixStats’, ‘RcppArmadillo’, ‘SparseM’, ‘MatrixModels’, ‘conquer’, ‘openxlsx’, ‘minqa’, ‘nloptr’, ‘RcppEigen’, ‘isoband’, ‘cpp11’, ‘carData’, ‘abind’, ‘pbkrtest’, ‘quantreg’, ‘maptools’, ‘rio’, ‘lme4’, ‘ggplot2’, ‘tidyr’, ‘broom’, ‘corrplot’, ‘car’, ‘ggrepel’, ‘ggsci’, ‘cowplot’, ‘ggsignif’, ‘gridExtra’, ‘polynom’, ‘rstatix’


Warning message in install.packages("ggpubr"):
“installation of package ‘nloptr’ had non-zero exit status”
Warning message in install.packages("ggpubr"):
“installation of package ‘lme4’ had non-zero exit status”
Warning message in install.packages("ggpubr"):
“installation of package ‘pbkrtest’ had non-zero exit status”
Warning message in install.packages("ggpubr"):
“installation of package ‘car’ had non-zero exit status”
Warning message in install.packages("ggpubr"):
“installation of package ‘rstatix’ had non-zero exit status”
Warning message in install.packages("ggpubr"):
“installation of package ‘ggpubr’ had non-zero 

In [1]:
library(stringr)
library(ggplot2)
library(ggpubr)

Warning message:
“package ‘ggpubr’ was built under R version 3.6.3”
Loading required package: magrittr



In [2]:
vdir=""#../from_list_01/
ndir=""#../fig2/
###IP
cfm="^I\\d+|^P\\d+|^AC\\d+"
md='0807'#'1105'#
nmd='0808'#'1105'#
csp=read.csv(file=paste0(ndir,"cnt-sp-norm-",nmd,".csv"),header=T,sep=',',as.is=F)
nsp=csp[intersect(grep(cfm,csp$a), grep(cfm,csp$b)),]
nsp=cbind(nsp,ab=paste(nsp[,'a'],nsp[,'b'],sep=","),
          ahost=NA,bhost=NA,shost=F,length=0)
spl=read.csv(file=paste0(vdir,"viruslist-single-pan",md,".tsv"),header=T,sep='\t',as.is=F)
colnames(spl)[which(colnames(spl)=="Host_lineage")]="EukTree"
nspl=spl[grep(cfm,spl$Abre),c('Abre','Length','EukTree','Family','aanum','gf.unique.fm','gf.shared')]
nsp=nsp[intersect(which(nsp[,'a'] %in% nspl[,'Abre']),which(nsp[,'b'] %in% nspl[,'Abre'])),]
# as.numeric(sub("I","",nspl$Abre[tail(grep('^I\\d+',nspl$Abre),1)]))

change=cbind(old=c(paste0("AC0",1:length(grep("^AC\\d+",nspl$Abre)))),
  new=c(paste0("I",(length(grep('^I\\d+',nspl$Abre))+1):((length(grep('^I\\d+',nspl$Abre))+1)+length(grep("^AC\\d+",nspl$Abre))-1))))

for(i in 1:nrow(change)){
  levels(nspl[,'Abre'])[which(levels(nspl[,'Abre'])==change[i,'old'])]=change[i,'new']
  levels(nsp[,'a'])[which(levels(nsp[,'a'])==change[i,'old'])]=change[i,'new']
  levels(nsp[,'b'])[which(levels(nsp[,'b'])==change[i,'old'])]=change[i,'new']
}

nsp=nsp[-which(str_extract(nsp[,'a'],"\\D+")==str_extract(nsp[,'b'],"\\D+")),]




for(i in 1:nrow(nsp)){
  nsp[i,'ahost']=as.character(nspl[grep(nsp[i,'a'],nspl[,'Abre']),'EukTree'])
  nsp[i,'bhost']=as.character(nspl[grep(nsp[i,'b'],nspl[,'Abre']),'EukTree'])
  nsp[i,'length']=sqrt(as.numeric(nspl[grep(nsp[i,'a'],nspl[,'Abre']),'Length'])*
    as.numeric(nspl[grep(nsp[i,'b'],nspl[,'Abre']),'Length']))
  nsp[i,'aamean']=sqrt(as.numeric(nspl[grep(nsp[i,'a'],nspl[,'Abre']),'aanum'])*
    as.numeric(nspl[grep(nsp[i,'b'],nspl[,'Abre']),'aanum']))
  nsp[i,'gpmean']=sqrt((as.numeric(nspl[grep(nsp[i,'a'],nspl[,'Abre']),'gf.unique.fm'])+
    as.numeric(nspl[grep(nsp[i,'a'],nspl[,'Abre']),'gf.shared']))*
      (as.numeric(nspl[grep(nsp[i,'b'],nspl[,'Abre']),'gf.unique.fm'])+
        as.numeric(nspl[grep(nsp[i,'b'],nspl[,'Abre']),'gf.shared'])))
  nsp[i,grep("ptera",nsp[i,])]="Insect"
  nsp[i,grep("Fish|ia$|Aves",nsp[i,])]="Vertebrate"
  if(nsp[i,'ahost']==nsp[i,'bhost']){nsp[i,'shost']=T}
  else{nsp[i,'shost']=F}
}

simhost=nsp[-which(nsp[,'shost']==F),]
sss=nsp[which(nsp[,'shost']==T),]
sss=unique(sss[,'ahost']) # shared hosts

#setdiff(which(nsp[,'shost']==F),intersect(which(nsp[,'ahost'] %in% sss),which(nsp[,'bhost'] %in% sss)))
difhost=nsp[setdiff(which(nsp[,'shost']==F),intersect(which(nsp[,'ahost'] %in% sss),which(nsp[,'bhost'] %in% sss))),]
simhost.l=c(as.character(unique(simhost[,'a'])),as.character(unique(simhost[,'b'])))

tdifhost=difhost[intersect(which(difhost[,'a'] %in% simhost.l),which(difhost[,'b'] %in% simhost.l)),]


In [3]:
pdf(paste0('P-I-',md,'.pdf'),
    width=8.268/2.1,height=11.693/4.2,paper='special')
pval=NULL; pvind=1
for(ht in sss){
  alist=nsp[union(which(nsp[,'ahost']==ht),which(nsp[,'bhost']==ht)),]
  simhost=alist[-which(alist[,'shost']==F),]
  difhost=alist[-which(alist[,'shost']==T),]
  
  gpval='gpnorm'
  pval[pvind]=wilcox.test(difhost[,gpval],simhost[,gpval],mu=0,alternative="less",conf.int=T)$p.value; pvind=pvind+1

  vio=ggplot(alist, aes(x=shost,y=alist[,gpval]))+
    geom_violin()+theme_linedraw()+geom_boxplot(width=0.05,outlier.shape = NA)+ #,outlier.shape = NA
    #geom_dotplot(binaxis='y', stackdir='center', dotsize=0.1)+#geom_jitter(width=0.25,size=0.5)+
    theme(axis.text.x=element_text(size=8),
      axis.text.y=element_text(size=10), axis.title=element_text(size=10))+
    scale_x_discrete(labels=c(paste0(ht," & non-",ht),paste0(ht,' & ',ht)))+
    labs(x=paste0(length(which(!alist$shost)),"  ",length(which(alist$shost))), 
      y=paste0("Level of genes sharing"))+ #
    stat_compare_means(label.x=1.2,label.y=max(nsp[,gpval])) #aes(label=..p.signif..),

  print(vio);#print(s);print(d)

}
dev.off()


Warning message:
“`select_()` was deprecated in dplyr 0.7.0.
Please use `select()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”
Warning message:
“`tbl_df()` was deprecated in dplyr 1.0.0.
Please use `tibble::as_tibble()` instead.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”


png 
  2

In [4]:
## file processing
cfm="^MI\\d+|^PY\\d+|^MP\\d+|^PN\\d+|^ME\\d+"

csp=read.csv(file=paste0(ndir,"cnt-sp-norm-",nmd,".csv"),header=T,sep=',',as.is=F)
nsp=csp[intersect(grep(cfm,csp$a), grep(cfm,csp$b)),]
nsp=cbind(nsp,ab=paste(nsp[,'a'],nsp[,'b'],sep=","),
          ahost=NA,bhost=NA,shost=F,length=0)

spl=read.csv(file=paste0(vdir,"viruslist-single-pan",md,".tsv"),header=T,sep='\t',as.is=F)
colnames(spl)[which(colnames(spl)=="Host_lineage")]="EukTree"
nspl=spl[grep(cfm,spl$Abre),c('Abre','Length','EukTree','Family','aanum','gf.unique.fm','gf.shared')]

nspl=nspl[-grep("unknow",nspl[,'EukTree']),]
nsp=nsp[intersect(which(nsp[,'a'] %in% nspl[,'Abre']),which(nsp[,'b'] %in% nspl[,'Abre'])),]

change=cbind(old=c(as.character(nspl$Abre[grep("^MP\\d+",nspl$Abre)]),paste0("PN0",1:length(grep("^PN\\d+",nspl$Abre)))),
  new=c(paste0("MI",(max(as.numeric(sub("\\D+","",nspl$Abre[grep('^MI\\d+',nspl$Abre)])))+1):
    ((max(as.numeric(sub("\\D+","",nspl$Abre[grep('^MI\\d+',nspl$Abre)])))+1)+length(grep("^MP\\d+",nspl$Abre))-1)),
  paste0("PY",(max(as.numeric(sub("\\D+","",nspl$Abre[grep('^PY\\d+',nspl$Abre)])))+1):
    ((max(as.numeric(sub("\\D+","",nspl$Abre[grep('^PY\\d+',nspl$Abre)])))+1)+length(grep("^PN\\d+",nspl$Abre))-1))))



for(i in 1:nrow(change)){
  levels(nspl[,'Abre'])[which(levels(nspl[,'Abre'])==change[i,'old'])]=change[i,'new']
  levels(nsp[,'a'])[which(levels(nsp[,'a'])==change[i,'old'])]=change[i,'new']
  levels(nsp[,'b'])[which(levels(nsp[,'b'])==change[i,'old'])]=change[i,'new']
}

nsp=nsp[-which(str_extract(nsp[,'a'],"\\D+")==str_extract(nsp[,'b'],"\\D+")),]


for(i in 1:nrow(nsp)){
  nsp[i,'ahost']=as.character(nspl[grep(nsp[i,'a'],nspl[,'Abre']),'EukTree'])
  nsp[i,'bhost']=as.character(nspl[grep(nsp[i,'b'],nspl[,'Abre']),'EukTree'])
  nsp[i,'length']=sqrt(as.numeric(nspl[grep(nsp[i,'a'],nspl[,'Abre']),'Length'])*
    as.numeric(nspl[grep(nsp[i,'b'],nspl[,'Abre']),'Length']))
  nsp[i,'aamean']=sqrt(as.numeric(nspl[grep(nsp[i,'a'],nspl[,'Abre']),'aanum'])*
    as.numeric(nspl[grep(nsp[i,'b'],nspl[,'Abre']),'aanum']))
  nsp[i,'gpmean']=sqrt((as.numeric(nspl[grep(nsp[i,'a'],nspl[,'Abre']),'gf.unique.fm'])+
    as.numeric(nspl[grep(nsp[i,'a'],nspl[,'Abre']),'gf.shared']))*
      (as.numeric(nspl[grep(nsp[i,'b'],nspl[,'Abre']),'gf.unique.fm'])+
        as.numeric(nspl[grep(nsp[i,'b'],nspl[,'Abre']),'gf.shared'])))
  #nsp[i,grep("treboux|mamiello",nsp[i,])]="chlorophytes"
  #nsp[i,grep("bicosoec|raphido|pelagophy|macroalg",nsp[i,])]="stramenopiles"
  nsp[i,grep("phyceae|ales|coccus|monas",nsp[i,])]="algae"
  #"Treboux|mamiello|raphido|pelagophy|macroalg|haptophy"
  nsp[i,grep("Bicos|Choano|Eubod",nsp[i,])]="hetero_flg"
  #nsp[i,grep("haptophy",nsp[i,])]="haptophytes"
  nsp[i,grep("Tubuline|Discosea",nsp[i,])]="amoeba"
  if(nsp[i,'ahost']==nsp[i,'bhost']){nsp[i,'shost']=T}else{
    nsp[i,'shost']=F}
}

simhost=nsp[-which(nsp[,'shost']==F),]
sss=nsp[which(nsp[,'shost']==T),]
sss=unique(sss[,'ahost']) # shared hosts

#setdiff(which(nsp[,'shost']==F),intersect(which(nsp[,'ahost'] %in% sss),which(nsp[,'bhost'] %in% sss)))
difhost=nsp[setdiff(which(nsp[,'shost']==F),intersect(which(nsp[,'ahost'] %in% sss),which(nsp[,'bhost'] %in% sss))),]


In [5]:
pdf(paste0('MI-PY-',md,'.pdf'),
   width=8.268/2.1,height=11.693/4.2,paper='special')
pval=NULL; pvind=1
for(ht in sss){
  alist=nsp[union(which(nsp[,'ahost']==ht),which(nsp[,'bhost']==ht)),]
  simhost=alist[-which(alist[,'shost']==F),]
  difhost=alist[-which(alist[,'shost']==T),]
  
  gpval='gpnorm'
  pval[pvind]=wilcox.test(difhost[,gpval],simhost[,gpval],mu=0,alternative="less",conf.int=T)$p.value; pvind=pvind+1

  vio=ggplot(alist, aes(x=shost,y=alist[,gpval]))+
    geom_violin()+theme_linedraw()+geom_boxplot(width=0.05,outlier.shape = NA)+
    #geom_dotplot(binaxis='y', stackdir='center', dotsize=0.1)+#geom_jitter(width=0.25,size=0.5)+
    theme(axis.text.x=element_text(size=8),
      axis.text.y=element_text(size=10), axis.title=element_text(size=10))+
    scale_x_discrete(labels=c(paste0(ht," & non-",ht),paste0(ht,' & ',ht)))+
    labs(x=paste0(length(which(!alist$shost)),"  ",length(which(alist$shost))), 
      y=paste0("Level of gene sharing"))+ #
    stat_compare_means(label.x=1.2,label.y=max(nsp[,gpval]))

  print(vio);#print(s);print(d)


}
dev.off()

png 
  2